In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[["Gender_Male","Age",'EstimatedSalary']]
dep=dataset[["Purchased"]]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
param_grid={'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], 'force_alpha':[True,False], 'binarize': [0.0], 'fit_prior':[True,False], 'class_prior':[None]}
grid= GridSearchCV(BernoulliNB(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda3run\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0],
                         'binarize': [0.0], 'class_prior': [None],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [12]:
y_pred=grid.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[64, 21],
       [ 9, 40]], dtype=int64)

In [14]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [15]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.88      0.75      0.81        85
           1       0.66      0.82      0.73        49

    accuracy                           0.78       134
   macro avg       0.77      0.78      0.77       134
weighted avg       0.80      0.78      0.78       134



In [16]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.8474189675870348

In [17]:
print ( " the best parameter is ", grid.best_params_)

 the best parameter is  {'alpha': 0.1, 'binarize': 0.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}


In [18]:
grid.cv_results_

{'mean_fit_time': array([0.0171535 , 0.00797811, 0.00757875, 0.00578442, 0.00498452,
        0.00578632, 0.00558491, 0.00558548, 0.00518217, 0.00458813,
        0.00498686, 0.00578547, 0.00498624, 0.00598483, 0.00758009,
        0.0063839 , 0.00518675, 0.00458813, 0.00498395, 0.00638175,
        0.00877705, 0.00578413, 0.00478668, 0.00558553, 0.00638337,
        0.00658035, 0.00957384, 0.00618286, 0.00618181, 0.00678186,
        0.00518689, 0.00578914, 0.00737734, 0.00677929, 0.00498638,
        0.00478821, 0.0053853 , 0.00418839, 0.00558558, 0.00658302]),
 'std_fit_time': array([5.89644531e-03, 2.09178354e-03, 1.95400895e-03, 3.98708684e-04,
        2.69571190e-06, 1.16359910e-03, 7.96652375e-04, 1.19727471e-03,
        1.93681026e-03, 7.98583933e-04, 1.54501837e-03, 1.71581730e-03,
        6.29923100e-04, 2.27311779e-03, 4.25956449e-03, 1.49175065e-03,
        9.76853604e-04, 7.97510354e-04, 1.09573057e-03, 2.86404007e-03,
        4.65225308e-03, 1.59667143e-03, 7.46595382e-04, 1.737

In [19]:
re=grid.cv_results_

In [20]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017154,0.005896,0.013764,1.162401e-03,0.1,0.0,None,True,True,"{'alpha': 0.1, 'binarize': 0.0, 'class_prior':...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,21
1,0.007978,0.002092,0.011369,1.017785e-03,0.1,0.0,None,True,False,"{'alpha': 0.1, 'binarize': 0.0, 'class_prior':...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,21
2,0.007579,0.001954,0.011171,7.463530e-04,0.1,0.0,None,False,True,"{'alpha': 0.1, 'binarize': 0.0, 'class_prior':...",0.726166,0.686880,0.667268,0.759244,0.778179,0.723547,0.041899,1
3,0.005784,0.000399,0.011768,1.595825e-03,0.1,0.0,None,False,False,"{'alpha': 0.1, 'binarize': 0.0, 'class_prior':...",0.726166,0.686880,0.667268,0.759244,0.778179,0.723547,0.041899,1
4,0.004985,0.000003,0.012565,1.019778e-03,0.2,0.0,None,True,True,"{'alpha': 0.2, 'binarize': 0.0, 'class_prior':...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,21
5,0.005786,0.001164,0.012165,1.719272e-03,0.2,0.0,None,True,False,"{'alpha': 0.2, 'binarize': 0.0, 'class_prior':...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,21
6,0.005585,0.000797,0.011766,1.160304e-03,0.2,0.0,None,False,True,"{'alpha': 0.2, 'binarize': 0.0, 'class_prior':...",0.726166,0.686880,0.667268,0.759244,0.778179,0.723547,0.041899,1
7,0.005585,0.001197,0.011169,1.163146e-03,0.2,0.0,None,False,False,"{'alpha': 0.2, 'binarize': 0.0, 'class_prior':...",0.726166,0.686880,0.667268,0.759244,0.778179,0.723547,0.041899,1
8,0.005182,0.001937,0.012566,1.847947e-03,0.3,0.0,None,True,True,"{'alpha': 0.3, 'binarize': 0.0, 'class_prior':...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,21
9,0.004588,0.000799,0.011170,1.163326e-03,0.3,0.0,None,True,False,"{'alpha': 0.3, 'binarize': 0.0, 'class_prior':...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,21


In [21]:
grid.best_score_

0.7235473441626866

In [24]:
pre_input= sc.transform([[0,35,60000]])
pre_input

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.99250926, -0.27290378, -0.27655878]])

In [25]:
result= grid.predict(pre_input)
result

array([0], dtype=int64)